# Bonus: Temperature Analysis I

In [1]:
import matplotlib.pyplot as plt
from matplotlib import style
style.use('fivethirtyeight')
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.types import Date
from sqlalchemy.orm import Session
from sqlalchemy import func
from sqlalchemy.sql.expression import extract
import pandas as pd 
from scipy.stats import ttest_ind
import numpy as np
 



In [2]:
# "tobs" is "temperature observations"
engine = create_engine("sqlite:///Resources/hawaii.sqlite")
conn = engine.connect()
session = Session(bind=engine)

In [3]:
Base = declarative_base()


In [4]:
# Convert the date column format from string to datetime
class Measurement(Base):
    __tablename__ = "measurement"
    
    id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(Date)
    prcp = Column(Float)
    tobs = Column(Float)

In [5]:
class Station(Base):
    __tablename__ = "station"
    
    id = Column(Integer, primary_key=True)
    station = Column(String)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation =  Column(Float)


In [6]:
# get all temperatures during June at all stations in all years

june_temps = session.query(Measurement.date, Measurement.tobs)\
    .filter(extract("month", Measurement.date) == "6").all()

In [7]:
# Set the date column as the DataFrame index
# convert to dataframe
june_temps = pd.DataFrame(june_temps, columns = ["Date", "Temperature"])

june_temps

,Date,Temperature
0,2010-06-01,78.0
1,2010-06-02,76.0
2,2010-06-03,78.0
3,2010-06-04,76.0
4,2010-06-05,77.0
...,...,...
1695,2017-06-26,79.0
1696,2017-06-27,74.0
1697,2017-06-28,74.0
1698,2017-06-29,76.0


### Compare June and December data across all years 

In [9]:
from scipy import stats

In [10]:
# Filter data for desired months
# get the average temperature in December at all stations in all years

dec_temps = session.query(Measurement.date, Measurement.tobs)\
    .filter(extract("month", Measurement.date) == "12").all()


In [11]:
# convert to dataframe
dec_temps = pd.DataFrame(dec_temps, columns = ["Date", "Temperature"])

dec_temps

,Date,Temperature
0,2010-12-01,76.0
1,2010-12-03,74.0
2,2010-12-04,74.0
3,2010-12-06,64.0
4,2010-12-07,64.0
...,...,...
1512,2016-12-27,71.0
1513,2016-12-28,71.0
1514,2016-12-29,69.0
1515,2016-12-30,65.0


In [12]:
# Identify the average temperature for June
june_temps = pd.DataFrame(june_temps, columns = ["Date", "Temperature"])

june_temps

,Date,Temperature
0,2010-06-01,78.0
1,2010-06-02,76.0
2,2010-06-03,78.0
3,2010-06-04,76.0
4,2010-06-05,77.0
...,...,...
1695,2017-06-26,79.0
1696,2017-06-27,74.0
1697,2017-06-28,74.0
1698,2017-06-29,76.0


In [13]:
# Identify the average temperature for December
# get average December temperature
dec_avg = dec_temps["Temperature"].mean()
dec_avg

71.04152933421226

In [14]:
# Create collections of temperature data
june = june_temps["Temperature"]

dec = dec_temps["Temperature"]

In [15]:
# Run paired t-test
results = ttest_ind(june, dec)

results

Ttest_indResult(statistic=31.60372399000329, pvalue=3.9025129038616655e-191)

### Analysis

T-test:
Above, a paired t-test was used to calculate the means of two different independent samples: temperature for June vs temperature for December across all available years in the dataset.

Results: t = 31.60, p = 3.90

Concluion: With the p-value exceeding the 0.05 level of significance, there is not sufficient evidence to conclude that there is a significant difference in means between June and December temperatures across all years available.
